# The Architecture format
A complete FPGA is specified in an architecture XML file and is wrapped in an `<architecture>` tag. `v8_example_arch.xml` defines a simple FPGA.

### [`<models>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#recognized-blif-models-models)
The first element in the architecture is `<models>`, which describes `blif` circuit models that the FPGA uses. The models  `.names`, `.latch`, `.input`, and `.output` are automatically recognized and don't need to be specified in this section. The `v8_example_arch.xml` architecture is simple enough to not need any additional models specified here.


### [`<layout>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#fpga-grid-layout)
The `<layout>` element specifies how the FPGA grid will be arranged. In `v8_example_arch.xml`: `<layout auto="1.000000"/>` specifies an automatic grid with an aspect ratio of 1.0. A specific width and heigh could be specified instead of the automatic layout. The rest of the `<layout>` section specifies that the perimeter will have `io` blocks, the corners will be `EMPTY` and the rest of the FPGA will be filled with `clb` blocks. Higher number priorities take precedence over lower numbers, so the `EMPTY` blocks in the corners will take precedence over the `io` blocks around the perimeter. The lowest priority `clb` blocks fill the remaining places in the grid.

`io` and `clb` blocks are defined in the `<complexblocklist>` section.

### [`<device>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#fpga-device-information)
The `<device>` element characterizes the transistors and connections of the FPGA. `<sizing>` specifies the resistance of the minimum-width nmos and pmos transistors. `<area grid_logic_tile_area>` is used as an estimate of the size of one grid tile.

The `<chan_width_distr>` section sets the relative widths of the routing channels in various parts of the FPGA. Here, all channels are set to be distributed uniformly. 

`<switch_block type="wilton" fs="3"/>` indicates a [Wilton switch block pattern](http://docs.verilogtorouting.org/en/latest/references/#wilton-phd) with a  switchblock fraction (fs) of 3.

`<connection_block input_switch_name="ipin_cblock"/>` specifies the name of the connection block. The actual connection block is defined in the `<switchlist>` section.

### [`<switchlist>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#switches)
The `<switchlist>` section specifies the switches used to connect wires and pins together. The switch names defined here are used elsewhere in the `<connection_block>` and `<segmentlist>` sections in this example architecture.


### [`<segmentlist>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#wire-segments)
`<segmentlist>` defines the wire segments in the channels between logic blocks. The `<segment>` element specifies one kind of wire to be used. `freq` is used to indicate the the usage ratio of this kind of wire with respect to all the wires. `length` is how many logic blocks the wire segment will span. `Rmetal` and `Cmetal` indicate the resistance and capacitance of the segment.

`<mux name>` is the name of the switch used to drive the segment and is defined in the `<switchlist>` section.

`<sb type>` is the pattern of switch blocks on the segment, and because switch blocks are between logic blocks there are one more entries in the list than the length of the segment. So for the segment with a length of 4, there are 5 entries in the pattern. `<cb type>` is the pattern of connection blocks on the segment. Connection blocks are next to logic blocks so the number of entries in the pattern equals the length of the segment.


## [`<complexblocklist>`](http://docs.verilogtorouting.org/en/latest/arch/reference/#complex-blocks)


# Skip I/O for now...


## Logic Blocks

Line 115 is where the general purpuse *complex logic block* or *clb* definition begins. `<pb_type>` is the tag to define a physical block on the FPGA. Next, the inputs and outputs to the block are defined; here there are 10 inputs, 4 outputs, and 1 clock input. The inputs and outputs have `equivalent=true` attributes, which means that they are logically equivalent and so order doesn't matter when routing.

Line 121 defines the *basic logic element* or *BLE* that makes up the clb. The `<pb_type name="fle" num_pb="4">` attribute indicates that 4 of these BLEs called `fle` are contained in the surrounding `clb`. The next lines define 4 inputs, one output, and one clock line.

Skipping down to line 134, the core lookup-table of the BLE is defined: `<pb_type name="lut4" blif_model=".names" num_pb="1" class="lut">`.  This indicates 1 lookup-table named `lut4`; `.names` is the BLIF keyword for lookup-table. The next two lines show 4 inputs and 1 output to the LUT. The `<delay_matrix>` attribute specifies the propogation delay through the LUT's inputs to output.

Line 147 defines the flip-flop in the BLE: `<pb_type name="ff" blif_model=".latch" num_pb="1" class="flipflop">`: 1 flip-flop named `ff`; `.latch` is the BLIF keyword for flip-flop. The next lines specify the I/Os and timing parameters.

Skipping back up to line 126, a mode named `n1_lut4` for the block named `fle` is defined. A block can have multiple modes specified, but a block can only use one mode at a time. This particular block only defines one mode. The mode defines a block on line 128 named `ble4` which contains the LUT `lut4` and flip-flop `ff`.

Moving back down to line 155, `<interconnect>` indicates how the blocks are connected. The `<direct>` element means to simply wire the nets together. Line 156 wires the input of `ble4` to the input of `lut4`, line 157 wires the output of `lut4` to the D-input of `ff`. Line 161 wires the clk inputs together. Line 162 defines a mux to set the output of `ble4` to be either the direct output of `lut4` or the latched output of `ff`.